In [1]:
# @title Default title text
import numpy as np

framework = 'tensorflow'

if framework == 'tensorflow':
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import SimpleRNN, Dense, Lambda

    timesteps = 10
    input_dim = 5
    output_dim = 1
    batch_size = 32

    X_train = np.random.randn(batch_size, timesteps, input_dim)
    y_train = np.random.randint(0, 2, size=(batch_size, output_dim))

    model = Sequential()
    model.add(Lambda(lambda x: tf.reverse(x, axis=[1]), input_shape=(timesteps, input_dim)))
    model.add(SimpleRNN(32))
    model.add(Dense(output_dim, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=8)

elif framework == 'pytorch':
    import torch
    import torch.nn as nn
    import torch.optim as optim

    timesteps = 10
    input_dim = 5
    hidden_size = 32
    output_dim = 1
    batch_size = 32

    X_train = torch.randn(batch_size, timesteps, input_dim)
    y_train = torch.randint(0, 2, (batch_size, output_dim)).float()

    class BackwardRNN(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(BackwardRNN, self).__init__()
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x):
            x = torch.flip(x, dims=[1])
            h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
            out, _ = self.rnn(x, h0)
            out = self.fc(out[:, -1, :])
            return torch.sigmoid(out)

    model = BackwardRNN(input_dim, hidden_size, output_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

else:
    print("Framework not recognized. Please choose 'tensorflow' or 'pytorch'.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.5583 - loss: 0.6394
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5625 - loss: 0.6120 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6250 - loss: 0.6288 
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6792 - loss: 0.5990
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7083 - loss: 0.5855
